In [1]:
{1,3}#Definindo finais de semana
mes = "04"
sabado = "22/"+mes+"/2023"
domingo = "23/"+mes+"/2023"

In [2]:
#PACOTES
import pandas as pd #manipulação de dados
import numpy as np #matematica

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import unidecode #arrumar strings

import re #regex

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
#lendo arquivo txt e transformando em dataframe cada mensagem
def read_file(file):
    '''Reads Whatsapp text file into a list of strings'''
    x = open(file, 'r', encoding = 'utf-8')
    y = x.read()
    content = y.splitlines()
    return content

chat = read_file('_chat.txt')
df = pd.DataFrame({'chat':chat})
df

,chat
0,[01/07/2017 09:50:51] CMA - Controle Interno: ...
1,[01/07/2017 09:50:51] Otto Mittermayer: ‎Otto...
2,[04/05/2022 10:17:06] CMA - Controle Interno: ...
3,[04/05/2022 10:17:17] Renata Mazzoni: ‎Otto M...
4,[04/05/2022 10:48:10] Rafael Tadeu Marques: 04...
...,...
33273,[17/04/2023 08:05:22] Gabriel Gondim Fonseca: ...
33274,[17/04/2023 08:24:49] Gabriel de Aguiar Santan...
33275,[17/04/2023 09:14:46] Fernanda Teles Ribeiro: ...
33276,[17/04/2023 09:28:05] Thassio Cunha De Santa: ...


# CAPTURANDO INFORMAÇÕES:

In [4]:
#DIA QUE FOI ENVIADA A MENSAGEM
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
df['dia'].unique()

array(['01/07/2017', '04/05/2022', 'afael tade', ..., '16/04/2023',
       'D3 Itaim)', '17/04/2023'], dtype=object)

In [5]:
#NOME DO ANESTESISTA
nome = []
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome.append(unidecode.unidecode(df['chat'][i][loc+1:loc+loc2].upper()))
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))
df['nome'] = nome
df['nome'].unique()

array(['CMA - CONTROLE INTERNO', 'OTTO  MITTERMAYER', 'RENATA MAZZONI',
       'RAFAEL TADEU MARQUES', '', 'AIDA', 'THIAGO SILVA PINTO',
       'RONALD FIGUEIREDO GOMES', 'KAIE FERNANDES',
       'GABRIEL DE AGUIAR SANTANA', 'FABIO ANTONIO DE SOUZA OLIVEIRA',
       'FABIO YUJI SUGUITA', 'PEDRO PAULO SILVA FARAH', 'NTRADA 4',
       'AIDA 13', 'SERGIO ZAMITH', 'AIDA 15', 'FELIPE DE OLIVEIRA NUNES',
       'FABIANA SOARES GAMA', 'ABIANA GAMA SAIDA AS 15',
       'PAULA ANGELICA PINHA NEVES', 'ARTHUR OLIVEIRA FACCHINI',
       'CLAUDIO MUELLER KAKUDA', 'MARCELO DOS ANJOS REIS',
       'SHEILA ITIROCO MONTE', 'SILVIA ALINE DOS SANTOS ANDRADE',
       'MANOEL DE SOUZA NETO', 'MARIANA DARDENGO BORGES',
       'GUSTAVO MARQUES DA SILVA', 'ESTANRLEY BARCELOS PINTO',
       'ALVARO LAZARO RAO REGONATO', 'BRUCE YE MAN CHOW', 'AIDA 17',
       'RICARDO KAWAURA', 'RODRIGO ACHUI HAGA', 'LEANDRO KENDI KANADA',
       'DANIELLEN LINS LOURENCO', 'JOSE CARLOS TUNA SCARPA', 'NTRADA 6',
       'AIDA 19'

In [6]:
#MENSAGEM ESCREVIDA
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'].unique()

array(['\u200eAs mensagens e as chamadas são protegidas com a criptografia de ponta a ponta e ficam somente entre você e os participantes desta conversa. Nem mesmo o WhatsApp pode ler ou ouvi-las.',
       '\u200eOtto  Mittermayer criou este grupo',
       '\u200eOtto  Mittermayer adicionou você', ...,
       '17/04 Itaim - Fernanda dispensada',
       '17/04 Jabaquara - Thassio dispensado (pos-Osasco)',
       '17/04 Osasco - alexandre Amaral dispensado (pós-Osasco)'],
      dtype=object)

In [7]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'

#salvando
df_prob = df[df['controle'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'].unique()

data probs: 3482
datas ok: 29796


array(['04/05', '4/5', '04/5', '05/05', '5/5', '05/5', '06/05', '6/5',
       '06/5', '07/05', '7/5', '07/5', '08/05', '8/5', '09/05', '9/5',
       '9/05', '10/05', '10/5', '11/05', '11/5', '12/05', '2/05', '12/06',
       '12/5', '13/05', '13/5', '3/05', '13/06', '14/05', '40/5', '14/5',
       '15/05', '16/05', '16/5', '17/05', '17/5', '17/06', '17/07',
       '18/05', '18/04', '18/5', '19/05', '19/5', '20/05', '20/5',
       '20/06', '21/05', '21/5', '21/04', '22/5', '22/05', '23/05',
       '23/04', '23/5', '24/05', '24/5', '25/05', '24/04', '25/5', '25/6',
       '26/05', '26/5', '26/06', '27/05', '27/5', '28/05', '28/5',
       '28/08', '29/05', '29/5', '30/05', '30/5', '03/05', '31/05',
       '31/5', '01/06', '1/6', '01/6', '1/06', '02/06', '2/6', '2/06',
       '02/05', '02/6', '03/06', '3/6', '03/6', '04/06', '4/6', '05/06',
       '5/6', '06/06', '6/6', '07/06', '7/6', '07/6', '07/09', '08/06',
       '08/07', '8/6', '08/6', '09/06', '9/6', '09/6', '10/06', '10/6',
       '

# Obs registros:

In [8]:
#CAPTURANDO UNIDADE
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0]
    else: #restante com nenhuma ou mais de uma unidade
        df['controle'][i] = 'PROB'
        
print(df['Q_unidades'].value_counts(),'\n')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'PROB']])
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

1    27557
2     1696
0      529
3       14
Name: Q_unidades, dtype: int64 

unidades probs: 5721
unidade ok: 27557


In [9]:
#RETIRANDO REGISTROS DE DESLOCADOS, DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE

#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

df['obs'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO|DISPENSADO|SAUDE|DESLOCADO|DISTANCIA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'PROB'
        
df['controle'].value_counts() 

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'PROB']])
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

unidades probs: 6392
unidade ok: 26886


In [10]:
#VERIFICANDO SE É EXTRA
df["extra"] = ''

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
df['extra'].value_counts()        

         24015
extra     2871
Name: extra, dtype: int64

# Tipo entrada e saida:

In [11]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'PROB'
    else: #restante com mais de uma informação
        df['controle'][i] = ''
        df['tipo'][i] = entradas_saidas[0]
        
print(df['tipo'].value_counts())        
print(df['controle'].value_counts())

df_prob = pd.concat([df_prob, df[df['controle'] == 'PROB']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

SAIDA      15152
ENTRADA     8995
            2739
Name: tipo, dtype: int64
        24147
PROB     2739
Name: controle, dtype: int64
probs: 9131
ok: 24147


# Data:

In [12]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [13]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['04/05', '04/5', '05/05', '05/5', '06/05', '06/5', '07/05', '07/5',
       '08/05', '08/5', '09/05', '09/5', '10/05', '10/5', '11/05', '11/5',
       '12/05', '12/06', '12/5', '13/05', '13/06', '14/05', '40/5',
       '14/5', '15/05', '16/05', '16/5', '17/05', '17/5', '18/05', '18/5',
       '19/05', '19/5', '20/05', '21/05', '21/5', '21/04', '22/05',
       '23/05', '23/04', '23/5', '24/05', '24/5', '24/04', '25/05',
       '25/5', '25/6', '26/05', '26/5', '26/06', '27/05', '27/5', '28/05',
       '28/5', '28/08', '29/05', '30/05', '30/5', '03/05', '31/05',
       '31/5', '01/06', '01/6', '02/06', '02/6', '02/05', '03/06', '03/6',
       '04/06', '04/6', '05/06', '05/6', '06/06', '06/6', '07/06', '07/6',
       '07/09', '08/06', '08/07', '08/6', '09/06', '09/6', '10/06',
       '11/06', '11/6', '13/6', '13/8', '14/06', '14/6', '15/06', '15/6',
       '16/06', '16/6', '17/06', '17/6', '18/06', '18/6', '19/06',
       '20/06', '20/6', '21/06', '21/6', '21/0', '22/06', '22/6', '23

In [14]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2023" 
    
print(df['data'].unique())

['04/04/2023' '05/04/2023' '06/04/2023' '07/04/2023' '08/04/2023'
 '09/04/2023' '10/04/2023' '11/04/2023' '12/04/2023' '13/04/2023'
 '14/04/2023' '40/04/2023' '15/04/2023' '16/04/2023' '17/04/2023'
 '18/04/2023' '19/04/2023' '20/04/2023' '21/04/2023' '22/04/2023'
 '23/04/2023' '24/04/2023' '25/04/2023' '26/04/2023' '27/04/2023'
 '28/04/2023' '29/04/2023' '30/04/2023' '03/04/2023' '31/04/2023'
 '01/04/2023' '02/04/2023' '60/04/2023' '37/04/2023']


In [15]:
df['sem_ou_fds'] = 'semana'

df.loc[(df['data'] == sabado) | (df['data'] == domingo), 'sem_ou_fds'] = 'fds'

df['sem_ou_fds'].value_counts()

semana    22585
fds        1562
Name: sem_ou_fds, dtype: int64

In [16]:
#df['data'].iloc[180] = '26/09/2022'

# Hora:

In [17]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [18]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'PROB'
        
print(df['controle'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'PROB']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

        23815
PROB      332
Name: controle, dtype: int64
probs: 9463
ok: 23815


In [19]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['11:10', '13:25', '13:34', '13:45', '14:45', '15:50', '16:10',
       '15:10', '16:05', '16:30', '16:50', '16:35', '06:00', '19:10',
       '19:15', '19:35', '20:10', '20:20', '20:37', '20:39', '20:05',
       '20:35', '20:50', '15:45', '22:10', '22:50', '22:45', '05:00',
       '04:20', '05:30', '05:35', '05:50', '06:40', '08:25', '09:30',
       '10:20', '11:40', '10:30', '14:00', '14:10', '17:00', '17:15',
       '19:45', '19:50', '20:00', '20:40', '21:00', '20:25', '22:18',
       '22:15', '22:20', '05:16', '05:20', '04:45', '05:15', '05:40',
       '06:25', '07:15', '06:10', '14:40', '15:20', '15:35', '15:40',
       '16:00', '16:15', '16:20', '16:55', '17:10', '17:59', '18:15',
       '19:20', '19:40', '21:05', '21:50', '05:25', '05:10', '05:45',
       '04:50', '10:10', '11:23', '11:15', '11:30', '12:57', '13:50',
       '14:25', '14:35', '15:28', '16:40', '16:45', '12:50', '17:45',
       '18:10', '18:45', '18:50', '18:40', '17:30', '19:00', '20:55',
       '21:30', '23:

# Hora final:

In [20]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [21]:
df['hora'].unique()

array(['11:10', '13:25', '13:34', '13:45', '14:45', '15:50', '16:10',
       '15:10', '16:05', '16:30', '16:50', '16:35', '06:00', '19:10',
       '19:15', '19:35', '20:10', '20:20', '20:37', '20:39', '20:05',
       '20:35', '20:50', '15:45', '22:10', '22:50', '22:45', '05:00',
       '04:20', '05:30', '05:35', '05:50', '06:40', '08:25', '09:30',
       '10:20', '11:40', '10:30', '14:00', '14:10', '17:00', '17:15',
       '19:45', '19:50', '20:00', '20:40', '21:00', '20:25', '22:18',
       '22:15', '22:20', '05:16', '05:20', '04:45', '05:15', '05:40',
       '06:25', '07:15', '06:10', '14:40', '15:20', '15:35', '15:40',
       '16:00', '16:15', '16:20', '16:55', '17:10', '17:59', '18:15',
       '19:20', '19:40', '21:05', '21:50', '05:25', '05:10', '05:45',
       '04:50', '10:10', '11:23', '11:15', '11:30', '12:57', '13:50',
       '14:25', '14:35', '15:28', '16:40', '16:45', '12:50', '17:45',
       '18:10', '18:45', '18:50', '18:40', '17:30', '19:00', '20:55',
       '21:30', '23:

In [22]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'PROB'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'PROB'
        else:
            df['controle'][i] = 'PROB'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'PROB'
        
df['hora'].unique()

[1900]
[1]
[1]
[11 13 14 15 16  6 19 20 22  5  4  8  9 10 17 21  7 18 12 23  0  2  1  3]


array(['11:30', '13:30', '14:00', '15:00', '16:00', '16:30', '15:30',
       '17:00', '06:00', '19:30', '20:00', '20:30', '21:00', '22:30',
       '23:00', '05:00', '04:00', '05:30', '06:30', '08:30', '09:30',
       '10:30', '12:00', '14:30', '17:30', '04:30', '07:30', '18:00',
       '18:30', '21:30', '22:00', '13:00', '19:00', '00:00', '08:00',
       '10:00', '11:00', '12:30', '23:30', '01:00', '09:00', '00:30',
       '02:00', '07:00', '02:30', '03:30', '01:30'], dtype=object)

# Setor:

In [23]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'PROB'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['ENTRADA'][i] = '13:00'
            df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'PROB'
    else:
        df['controle'][i] = 'PROB'
        
df['PERIODO'].value_counts()

print(df['controle'].value_counts())

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'PROB']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df))

        23676
PROB      139
Name: controle, dtype: int64
probs: 9602
ok: 23676


In [24]:
#DURAÇÃO DO PLANTÃO
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'].unique()

array(['04:30', '00:30', '01:00', '02:00', '03:00', '03:30', '02:30',
       '04:00', '01:30', '05:00', '05:30', '00:00', '06:00'], dtype=object)

In [25]:
df['setor'] = ''

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
df['setor'].value_counts()

SEMANA - 6HS                    7489
SEMANA - TARDE - C.CIRURGICO    6643
SEMANA - NOTURNO - SAIDA        5912
SEMANA - MANHA - C.CIRURGICO    2101
FDS - 6HS                        523
FDS - SAIDA                      480
FDS - PLANTAO                    403
FDS - C.CIRURGICO ROTINA         125
Name: setor, dtype: int64

In [26]:
print("probs:", len(df_prob))
print("ok:", len(df))

porcentagem = round((len(df)/(len(df)+len(df_prob)))*100, 2)
print('Porcentagem de registros corrigidas pelo o algoritmo:', porcentagem)

probs: 9602
ok: 23676
Porcentagem de registros corrigidas pelo o algoritmo: 71.15


# Ultimos ajustes:

In [27]:
#Distancias
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'PROB'

In [28]:
for i in range(0, len(df)):
    if df['DURAÇÃO'][i] == "00:00":
        df['controle'][i] = "PROB"
        
print(df['controle'].value_counts())

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'PROB']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df))

        23082
PROB      594
Name: controle, dtype: int64
probs: 10196
ok: 23082


In [29]:
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
df['unidade'] = df['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

df['unidade'] = df['unidade'].apply(lambda x: x.replace('JABA','JABAQUARA'))

df['unidade'] = df['unidade'].apply(lambda x: x.replace('SANTA','SANTA ISABEL'))

df['unidade'].unique()

#problemas com santa isabel 
#df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - SAIDA")] = "FDS - NOTURNO - SAIDA"
df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - C.CIRURGICO ROTINA")] = "FDS - MANHA - C.CIRURGICO"

In [30]:
#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABA','JABAQUARA'))
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))

#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABAQUARAQUARA','JABAQUARA'))
#df['UNIDADE'].unique()

# Salvando:

In [31]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'ENTRADA', 'SAIDA', 'DURAÇÃO', 'PERIODO', 'setor', 'sem_ou_fds']]

In [35]:
df.tail(49)

,chat,dia,nome,mensagem,controle,data,unidade,extra,ENTRADA,SAIDA,DURAÇÃO,PERIODO,setor,sem_ou_fds
23033,[16/04/2023 05:36:49] Fabio Yuji Suguita: 16/0...,16/04/2023,FABIO YUJI SUGUITA,16/04 aviccena yuji entrada 05:30,,16/04/2023,AVICCENA,,05:30,07:00,01:30,6h,SEMANA - 6HS,semana
23034,[16/04/2023 05:59:15] Daniellen Lins Lourenço:...,16/04/2023,DANIELLEN LINS LOURENCO,16/04 VNS - Daniellen entrada 06:00 ( D1 ),,16/04/2023,VNS,,06:00,07:00,01:00,6h,SEMANA - 6HS,semana
23035,[16/04/2023 06:13:06] Darlan Junior Schneider:...,16/04/2023,DARLAN JUNIOR SCHNEIDER,16/04 Morumbi - Darlan entrada 06:00,,16/04/2023,MORUMBI,,06:00,07:00,01:00,6h,SEMANA - 6HS,semana
23036,[16/04/2023 07:26:09] Yuri Lucas De Souza: 16/...,16/04/2023,YURI LUCAS DE SOUZA,16/04 Villa - Souza saída 07:15 ( pós plantão ),,16/04/2023,VILLA,,07:00,07:30,00:30,manha,SEMANA - MANHA - C.CIRURGICO,semana
23037,[16/04/2023 08:26:39] Diogo Candeo Rodrigues C...,16/04/2023,DIOGO CANDEO RODRIGUES CORDEIRO,16/04 Morumbi - Candeo entrada às 07:05 ( D2 ),,16/04/2023,MORUMBI,,07:00,13:00,06:00,manha,SEMANA - MANHA - C.CIRURGICO,semana
23038,[16/04/2023 08:51:09] Maxvilano Lima De Macedo...,16/04/2023,MAXVILANO LIMA DE MACEDO,16/04 Morumbi - Maxvilano entrada às 05:40 ( ...,,16/04/2023,MORUMBI,,05:30,07:00,01:30,6h,SEMANA - 6HS,semana
23039,[16/04/2023 09:04:24] Marina Pontes Moura Nasc...,16/04/2023,MARINA PONTES MOURA NASCIMENTO E SILVA,16/04 Villa - Moura entrada 05:45,,16/04/2023,VILLA,,05:30,07:00,01:30,6h,SEMANA - 6HS,semana
23040,[16/04/2023 09:16:44] Natalia Maia Queiroz: 16...,16/04/2023,NATALIA MAIA QUEIROZ,16/04 Analia- Queiroz entrada 06:40 ( D2 ),,16/04/2023,ANALIA,,06:30,07:00,00:30,6h,SEMANA - 6HS,semana
23041,[16/04/2023 10:21:38] Cláudio Mueller Kakuda: ...,16/04/2023,CLAUDIO MUELLER KAKUDA,16/04 Maternidade - Kakuda entrada 7:40,,16/04/2023,MATERNIDADE,,07:30,13:00,05:30,manha,SEMANA - MANHA - C.CIRURGICO,semana
23042,[16/04/2023 11:22:28] Caio Túlio Duarte: 16/04...,16/04/2023,CAIO TULIO DUARTE,16/04 Maternidade - Caio entrada 08:25,,16/04/2023,MATERNIDADE,,08:00,13:00,05:00,manha,SEMANA - MANHA - C.CIRURGICO,semana


In [ ]:
df.to_excel("df_gabs.xlsx", index=False)
df_prob.to_excel('df_giov.xlsx', index=False)

In [ ]:
df

In [ ]:
df.head(45)

In [ ]:
print(len(df))
print(len(df_prob))

# Plantões com problemas:

In [ ]:
print("Registros dos dias:", sorted(list(df_prob["dia"].unique())))
print("quantidade de registros:", len(df_prob), "=",
      str(round((len(df_prob)/len(df))*100, 2)) + "% de todos os registros feitos")
print("#"*30, "\n")

for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

# OUTROS: